In [ ]:
import numpy as np
import math

In [ ]:
#@title
def intersect3D_SegmentTriangle(triLst,segLst):
  '''
  This function calculates the intersection point of a triangle with a segment.

  Inputs:
      A list of three 3D points (tuples) representing the triangle
      A list of two 3D points (tuples) representing the segment

  Output:
      A 3D points (tuples) representing the intersection point
  '''
  SMALL_NUM = 1e-8
  V0,V1,V2 = np.array(triLst[0]),np.array(triLst[1]),np.array(triLst[2])
  P0,P1 = np.array(segLst[0]),np.array(segLst[1])
  u = V1 - V0
  v = V2 - V0
  n = np.cross(u,v)
  # n = u*v

  dir = P1 - P0 # segment direction vector
  w0 = P0 - V0
  a = -np.dot(n,w0)
  b = np.dot(n,dir)
  r = a/b
  # if r < 0 or r > 1:
  #   return('the segment goes away from triangle')
  
  # calculate the intersect point 
  I = P0 + r*dir
  return I

t = [(2000,0,0),(0,0,0),(0,2000,0)]
T = [(-0.254, -30.2, 4.396), (-5.302, -29.14, -0.002), (-2.09, -29.542, -0.002)]
s1 = [(-3.208, -30.156, 4.394), (-5.302, -29.14, -0.002)]
s2 = [(-5.302, -29.14, -0.002), (-0.254, -30.2, 4.396)]
I = intersect3D_SegmentTriangle(t,s1)
print(I)
print(type(I))


[ -5.30104732 -29.14046224   0.        ]
<class 'numpy.ndarray'>


In [ ]:
#@title
def getI(triLst,h):
  '''
  This function calculates the intersection of a triangle with a horizontal plane.

  Inputs:
      A list of three 3D points (tuples) representing P0, P1, P2 respectively.

  Output:
      A np.array of two 3D points (tuples) representing I1, I2
  '''
  P0,P1,P2 = triLst[0],triLst[1],triLst[2]
  # pt0z = P0[2]
  # pt2z = P2[2]
  triLst = ((0,0,h), (100,0,h), (0,100,h))
  # V0,V1,V2 = np.array(triLst[0]),np.array(triLst[1]),np.array(triLst[2])

  # After determined P0,P1,P2, calculate I1 and I2
  segLst1 = (P0,P1)
  segLst2 = (P0,P2)
  I1 = intersect3D_SegmentTriangle(triLst,segLst1)
  I2 = intersect3D_SegmentTriangle(triLst,segLst2)
  # print('I1: ', I1)
  # print('I2: ', I2)
  return(I1, I2)

T = [(-0.254, -30.2, 4.396), (-5.302, -29.14, -0.002), (-2.09, -29.542, -0.002)]
# I1,I2 = getI(T)
# print('I1: ', I1, 'I2: ', I2)
# print(type(I1[0]))

In [ ]:
#@title
def stl_reader(stlfile):
  '''
  This generator import a STL file and yield one line each time the user call it.

  Input:
      A STL file

  Output:
      A tuple of a 3D point
  '''
  with open(stlfile,'r') as file:
    for line in file:
      if line.strip().split(' ')[0] == 'vertex':
        yield line.strip().split()[1:]

def genTri(stlfile):
  '''
  This function import a STL file and convert it to a list of three 3D points,
  these points are the vertex of a triangle

  Input:
      A STL file

  Output:
      A list of three 3D points (tuples)
  '''
  lst = []
  stl_gen = stl_reader(stlfile)
  for line in stl_gen:
    lst.append(tuple(line))

  return lst

lst = genTri('twist_gear_vase2.stl')
# print(lst)
print(len(lst))
print('lst[-1]: ', lst[-1])
print('type(lst[-1]): ', type(lst[-1]))
print(float(lst[-1][2]))
print(type(float(lst[-1][2])))


9000
lst[-1]:  ('1.150000e+01', '4.800000e+00', '9.753400e+01')
type(lst[-1]):  <class 'tuple'>
97.534
<class 'float'>


In [ ]:

def genIlist(stlfile):
  '''
  This function takes a list of 3D points (tuples) and return a 
  list of I

  Input:
      A list of 3D points.

  Output:
      a list of I (tuples) representing the intersection point.
  '''
  tupleLst = genTri(stlfile)
  raw_I_list = []
  height = 0
  while height <= float(tupleLst[-1][2]):
    for i in range(0, len(tupleLst), 3):

      pt0x,pt1x,pt2x = float(tupleLst[i][0]),float(tupleLst[i+1][0]),float(tupleLst[i+2][0])
      pt0y,pt1y,pt2y = float(tupleLst[i][1]),float(tupleLst[i+1][1]),float(tupleLst[i+2][1])
      pt0z,pt1z,pt2z = float(tupleLst[i][2]),float(tupleLst[i+1][2]),float(tupleLst[i+2][2])
      # V0 = np.array([0, 0,   height])
      # V1 = np.array([10000, 0, height])
      # V2 = np.array([0, 10000, height])
      if pt0z <= height < pt2z or pt2z <= height < pt0z:
      # if current height is inside the current triangle
        if math.isclose(pt0z, pt1z, abs_tol=1e-2):
        # if point 2 is on one side alone
          P0 = (pt2x, pt2y, pt2z)
          P1 = (pt0x, pt0y, pt0z)
          P2 = (pt1x, pt1y, pt1z)
          I1,I2 = getI([P0, P1, P2], height)

        elif math.isclose(pt1z, pt2z, abs_tol=1e-2):
        # if point 0 is on one side alone
          P0 = (pt0x, pt0y, pt0z)
          P1 = (pt1x, pt1y, pt1z)
          P2 = (pt2x, pt2y, pt2z)
          I1,I2 = getI([P0, P1, P2], height)

      raw_I_list.append([I1,I2])
    height += 1

  return raw_I_list

lst = genIlist('twist_gear_vase2.stl')

print('len(lst): ', len(lst), 'type(lst): ', type(lst))
print(lst[:10])
print(lst[50])
print(type(lst[0]))
print(lst[0][0])
print(type(lst[0][0]))
print(lst[0][0][0])
print(type(lst[0][0][0]))

len(lst):  294000 type(lst):  <class 'list'>
[[array([-5.29970441e+00, -2.91404820e+01, -4.33680869e-19]), array([ -5.30104732, -29.14046224,   0.        ])], [array([ -5.29970441, -29.14048204,   0.        ]), array([ -2.08916508, -29.54229923,   0.        ])], [array([ -5.29970441, -29.14048204,   0.        ]), array([ -2.08916508, -29.54229923,   0.        ])], [array([ -8.38564331, -28.41479254,   0.        ]), array([ -5.30104732, -29.14046224,   0.        ])], [array([-1.13296004e+01, -2.73795299e+01, -4.33680869e-19]), array([-11.33146642, -27.37907172,   0.        ])], [array([-11.32960037, -27.37952995,   0.        ]), array([ -8.3869465 , -28.41505624,   0.        ])], [array([-11.32960037, -27.37952995,   0.        ]), array([ -8.3869465 , -28.41505624,   0.        ])], [array([-14.1161937 , -26.05567656,   0.        ]), array([-11.33146642, -27.37907172,   0.        ])], [array([-16.72604732, -24.46146133,   0.        ]), array([-16.72715196, -24.4608071 ,   0.        ])], 

In [ ]:
dist = np.sqrt((lst[0][0][0] - lst[0][1][0])**2 + (lst[0][0][1] - lst[0][1][1])**2)
print('dist: ', dist)

dist:  0.0013430505885283545


In [ ]:
import csv
with(open('Lab3.gcode', 'w')) as f:
    me = csv.writer(f)
    me.writerow(["M140 S60"])
    me.writerow(["M105"])
    me.writerow(["M190 S60"])
    me.writerow(["M104 S230"])
    me.writerow(["M105"])
    me.writerow(["M109 S230"])
    me.writerow(["M83"])
    me.writerow(["G92 E0"])
    me.writerow(["G28"])
    me.writerow(["G91"])
    me.writerow(["G1 Z2.0 F3000"])
    me.writerow(["G1 X5 Y20 Z0.3 F5000.0"])
    me.writerow(["G92 E0"])
    me.writerow(["G90"])
    me.writerow(["G1 F2700 E-5"])
    me.writerow(["M107"])
    me.writerow(["G1 F1200 E0"])
    E = 0.2  
    for i in range(len(lst)):
      dist = np.sqrt((str(lst[i][0][0]) - str(lst[i][1][0]))**2 + (str(lst[i][0][1]) - str(lst[i][1][1]))**2)
      E = dist * 0.0332569
      me.writerow(["G1 " + "X" + str(lst[i][0][0]) + " Y" + str(lst[i][0][1])  + " Z" + str(lst[i][0][2]) + " E" + str(E)])
      me.writerow(["G1 " + "X" + str(lst[i][1][0]) + " Y" + str(lst[i][1][1])  + " Z" + str(lst[i][1][2]) + " E" + str(E)])
      E += 0.05
    me.writerow(["G0 Z10"])
    me.writerow(["G28"])

In [ ]:
dist = np.sqrt((lst[0][0][0] - lst[0][1][0])**2 + (lst[0][0][1] - lst[0][1][1])**2)

In [ ]:
#@title
def divide_by_plane(raw_L_list):
'''
This function divide the raw L list according to their heights. 
Segments will be in a same list if they belong to a same plane.
'''

In [ ]:
#@title
def sorting(L_list_before_order):
  '''
  This function sorts a list of unordered segments into a list of 
  ordered segments 

  Input:
      A list of line segments, each line segment is a list of 2D 
      points (tuples).

  Output:
      a single list of 2D points where the start of one segment is
      the end of another.
  '''
  # first, find out which segment is the start
  for 



  # then, according the starting segment, sorting the rest segments

  return 0

In [ ]:
# P0 = (1, 1, 1)
# P1 = (2, 2, 2)
# P2 = (3, 3, 3)
# triLst = [P0, P1, P2]
T = (-5.302000e+00, -2.914000e+01, -2.000000e-03)
print(T)
print(type(T))
print(type(T[0]))


(-5.302, -29.14, -0.002)
<class 'tuple'>
<class 'float'>
